# MultiCollinearity
### In multiple linear regression, there may be strong correlations between the explanatory variables. Which can lead to complications and prediction problems
### We want to keep our modelsl parsimonious and use the features that explain most of the variation
### Strong correlation between variables may make a matrix non invertible
### We can look at the correlation matrix to see the extent of correlation between the combinations of  features
### And then we have a number of options available to identify the best features that would lead to a robust model
### We could use variance_inflation_factor from statsmodels and eliminate features having a correlation above a certain threshold
### We can use RFE (Recursive Featue Elimination) from featur_selection to limit the number of features and compare performance 
### We can use pca to reduce the number of features though we should consciously try to limit its use to data size management rather than employing it as a feature pruning mechanism

In [80]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.linear_model import OLS

In [17]:
?variance_inflation_factor

In [9]:
# lets load the boston house prices dataset to work the concepts outv
from sklearn.datasets import load_boston
boston = load_boston()

In [10]:
# lets review the dataset
print(boston.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [11]:
# Lets create the design matrix X and the target variable y
X = boston.data
y = boston.target

In [12]:
# check the shape
X.shape, y.shape

((506, 13), (506,))

In [13]:
boston.feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'],
      dtype='<U7')

In [14]:
# Lets use corrcoef from numpy to build a correlation matrix between features analogous to the corr function 
# available in R
# Here we wrap it in a data frame for ease of perusal
boston_corr_df = pd.DataFrame(np.corrcoef(np.transpose(X)),
                              columns=boston.feature_names, 
                              index=boston.feature_names)

In [43]:
boston_corr_df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
CRIM,1.000000,-0.199458,0.404471,-0.055295,0.417521,-0.219940,0.350784,-0.377904,0.622029,0.579564,0.288250,-0.377365,0.452220
ZN,-0.199458,1.000000,-0.533828,-0.042697,-0.516604,0.311991,-0.569537,0.664408,-0.311948,-0.314563,-0.391679,0.175520,-0.412995
INDUS,0.404471,-0.533828,1.000000,0.062938,0.763651,-0.391676,0.644779,-0.708027,0.595129,0.720760,0.383248,-0.356977,0.603800
CHAS,-0.055295,-0.042697,0.062938,1.000000,0.091203,0.091251,0.086518,-0.099176,-0.007368,-0.035587,-0.121515,0.048788,-0.053929
NOX,0.417521,-0.516604,0.763651,0.091203,1.000000,-0.302188,0.731470,-0.769230,0.611441,0.668023,0.188933,-0.380051,0.590879
RM,-0.219940,0.311991,-0.391676,0.091251,-0.302188,1.000000,-0.240265,0.205246,-0.209847,-0.292048,-0.355501,0.128069,-0.613808
AGE,0.350784,-0.569537,0.644779,0.086518,0.731470,-0.240265,1.000000,-0.747881,0.456022,0.506456,0.261515,-0.273534,0.602339
DIS,-0.377904,0.664408,-0.708027,-0.099176,-0.769230,0.205246,-0.747881,1.000000,-0.494588,-0.534432,-0.232471,0.291512,-0.496996
RAD,0.622029,-0.311948,0.595129,-0.007368,0.611441,-0.209847,0.456022,-0.494588,1.000000,0.910228,0.464741,-0.444413,0.488676
TAX,0.579564,-0.314563,0.720760,-0.035587,0.668023,-0.292048,0.506456,-0.534432,0.910228,1.000000,0.460853,-0.441808,0.543993


In [44]:
boston_corr_df[lambda x: x > 0.05]

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
CRIM,1.000000,NaN,0.404471,NaN,0.417521,NaN,0.350784,NaN,0.622029,0.579564,0.288250,NaN,0.452220
ZN,NaN,1.000000,NaN,NaN,NaN,0.311991,NaN,0.664408,NaN,NaN,NaN,0.175520,NaN
INDUS,0.404471,NaN,1.000000,0.062938,0.763651,NaN,0.644779,NaN,0.595129,0.720760,0.383248,NaN,0.603800
CHAS,NaN,NaN,0.062938,1.000000,0.091203,0.091251,0.086518,NaN,NaN,NaN,NaN,NaN,NaN
NOX,0.417521,NaN,0.763651,0.091203,1.000000,NaN,0.731470,NaN,0.611441,0.668023,0.188933,NaN,0.590879
RM,NaN,0.311991,NaN,0.091251,NaN,1.000000,NaN,0.205246,NaN,NaN,NaN,0.128069,NaN
AGE,0.350784,NaN,0.644779,0.086518,0.731470,NaN,1.000000,NaN,0.456022,0.506456,0.261515,NaN,0.602339
DIS,NaN,0.664408,NaN,NaN,NaN,0.205246,NaN,1.000000,NaN,NaN,NaN,0.291512,NaN
RAD,0.622029,NaN,0.595129,NaN,0.611441,NaN,0.456022,NaN,1.000000,0.910228,0.464741,NaN,0.488676
TAX,0.579564,NaN,0.720760,NaN,0.668023,NaN,0.506456,NaN,0.910228,1.000000,0.460853,NaN,0.543993


In [84]:
from sklearn.linear_model import LinearRegression
flr = LinearRegression()
fn_lr_model = flr.fit(boston.data, boston.target)
fn_lr_model_score = fn_lr_model.score(boston.data, boston.target)
print(fn_lr_model_score)
totvar_boston = np.sum((boston.target - np.mean(boston.target)) ** 2)
print(totvar_boston)
fn_lr_rsqd = (1 - fn_lr_model_score) * totvar_boston
print(fn_lr_rsqd)

for i in range(13):
    print('\n',boston.feature_names[i])
    ncols = boston.data.shape[1]
    mask = np.arange(ncols) != i
    col_data_to_check = boston.data[:, i].reshape(boston.data.shape[0])
    sv_data = boston.data[:, mask]
    print(sv_data.shape)
    # sv_data = boston.data[:, bcv].reshape(boston.data.shape[0], 1)
    ptr_lr_model = flr.fit(sv_data, boston.target)
    ptr_lr_model_score = ptr_lr_model.score(sv_data, boston.target)
    print(ptr_lr_model_score)
#     print(OLS.fit())
#     ptr_lr_rsqd = (1 - ptr_lr_model_score) * totvar_boston
#     print(ptr_lr_rsqd)

0.740607742865
42716.295415
11080.2762841

 CRIM
(506, 12)
0.734948825334

 ZN
(506, 12)
0.734586009139

 INDUS
(506, 12)
0.740547079512

 CHAS
(506, 12)
0.735474359356

 NOX
(506, 12)
0.729169345743

 RM
(506, 12)
0.696926451754

 AGE
(506, 12)
0.74060603879

 DIS
(506, 12)
0.711753545546

 RAD
(506, 12)
0.729413596642

 TAX
(506, 12)
0.734941203971

 PTRATIO
(506, 12)
0.712610478535

 B
(506, 12)
0.734148774133

 LSTAT
(506, 12)
0.683952111911


In [45]:
# here we define a manual procedure to eliminate columns having correlation above a certain threshold
def multicollinearity_check(X, thresh=5.0):
    data_type = X.dtypes
    # print(type(data_type))
    int_cols = \
    X.select_dtypes(include=['int', 'int16', 'int32', 'int64', 'float', 'float16', 'float32', 'float64']).shape[1]
    total_cols = X.shape[1]
    try:
        if int_cols != total_cols:
            raise Exception('All the columns should be integer or float, for multicollinearity test.')
        else:
            variables = list(range(X.shape[1]))
            dropped = True
            print('''\n\nThe VIF calculator will now iterate through the features and calculate their respective values.
            It shall continue dropping the highest VIF features until all the features have VIF less than the threshold of 5.\n\n''')
            while dropped:
                dropped = False
                vif = [variance_inflation_factor(X.iloc[:, variables].values, ix) for ix in variables]
                print('\n\nvif is: ', vif)
                maxloc = vif.index(max(vif))
                if max(vif) > thresh:
                    print('dropping \'' + X.iloc[:, variables].columns[maxloc] + '\' at index: ' + str(maxloc))
                    # del variables[maxloc]
                    X.drop(X.columns[variables[maxloc]], 1, inplace=True)
                    variables = list(range(X.shape[1]))
                    dropped = True

            print('\n\nRemaining variables:\n')
            print(X.columns[variables])
            # return X.iloc[:,variables]
            return X
    except Exception as e:
        print('Error caught: ', e)

In [46]:
# Lets create the boston panda data frame which we shall feed into multicollinearity_check to identify best features
boston_pd = pd.DataFrame(X, columns=boston.feature_names)

In [47]:
boston_pd

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


In [67]:
boston_pd_for_mcoll = boston_pd.copy()
# boston_pd_for_mcoll
multicollinearity_check(boston_pd_for_mcoll, 5.0)



The VIF calculator will now iterate through the features and calculate their respective values.
            It shall continue dropping the highest VIF features until all the features have VIF less than the threshold of 5.




vif is:  [2.0746257632525675, 2.8438903527570782, 14.48428343503152, 1.1528909172683364, 73.902211708121285, 77.934968671814261, 21.386773583047781, 14.699368125642422, 15.154741587164747, 61.226929320337554, 85.027313520427597, 20.06600706112129, 11.088865100659874]
dropping 'PTRATIO' at index: 10


vif is:  [2.0736651170637597, 2.4516148109419009, 14.273984566637807, 1.1420964084702536, 73.901444466440083, 60.578629936371108, 21.361196758126564, 12.222012071841089, 15.146039975593606, 59.301499402931668, 18.578773333710718, 10.123960834459329]
dropping 'NOX' at index: 4


vif is:  [2.0716753951147542, 2.4496712046454232, 13.149921335316749, 1.1382153060334725, 41.39222112923251, 19.889346207448238, 12.032864080018738, 15.142075468353935, 57.720177047564796, 18

,CRIM,ZN,CHAS,DIS,RAD,LSTAT
0,0.00632,18.0,0.0,4.0900,1.0,4.98
1,0.02731,0.0,0.0,4.9671,2.0,9.14
2,0.02729,0.0,0.0,4.9671,2.0,4.03
3,0.03237,0.0,0.0,6.0622,3.0,2.94
4,0.06905,0.0,0.0,6.0622,3.0,5.33
5,0.02985,0.0,0.0,6.0622,3.0,5.21
6,0.08829,12.5,0.0,5.5605,5.0,12.43
7,0.14455,12.5,0.0,5.9505,5.0,19.15
8,0.21124,12.5,0.0,6.0821,5.0,29.93
9,0.17004,12.5,0.0,6.5921,5.0,17.10


In [68]:
# the dataframe after being wrung through the multicollinearity check
boston_pd_for_mcoll

,CRIM,ZN,INDUS,CHAS,DIS,RAD,LSTAT
0,0.00632,18.0,2.31,0.0,4.0900,1.0,4.98
1,0.02731,0.0,7.07,0.0,4.9671,2.0,9.14
2,0.02729,0.0,7.07,0.0,4.9671,2.0,4.03
3,0.03237,0.0,2.18,0.0,6.0622,3.0,2.94
4,0.06905,0.0,2.18,0.0,6.0622,3.0,5.33
5,0.02985,0.0,2.18,0.0,6.0622,3.0,5.21
6,0.08829,12.5,7.87,0.0,5.5605,5.0,12.43
7,0.14455,12.5,7.87,0.0,5.9505,5.0,19.15
8,0.21124,12.5,7.87,0.0,6.0821,5.0,29.93
9,0.17004,12.5,7.87,0.0,6.5921,5.0,17.10


In [69]:
# lets employ RFE now
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
model = LinearRegression()
rfe = RFE(model, 8)
clf = rfe.fit(X, y)

In [71]:
boston.feature_names[clf.support_]

array(['CRIM', 'CHAS', 'NOX', 'RM', 'DIS', 'RAD', 'PTRATIO', 'LSTAT'],
      dtype='<U7')